In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8 # length of integers
batch_size = 4 # how many of blocks are running in parallel
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [15]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)

In [3]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([72, 34, 48, 45,  1, 37, 55, 54, 44, 45, 58, 46, 61, 52,  1, 37, 49, 66,
        41, 58, 44,  1, 55, 46,  1, 29, 66,  0,  0, 42, 65,  1, 26,  8,  1, 20,
        58, 41, 54, 51,  1, 16, 41, 61, 53,  0,  0,  0, 34, 48, 49, 59,  1, 42,
        55, 55, 51,  1, 49, 59,  1, 44, 45, 44, 49, 43, 41, 60, 45, 44,  1, 60,
        55,  1, 53, 65,  1, 47, 55, 55, 44,  1, 46, 58, 49, 45, 54, 44,  1,  3,
         1, 43, 55, 53, 58, 41, 44, 45,  0, 27])


In [4]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]
print(train_data)
print(val_data)

tensor([72, 34, 48,  ...,  8, 71,  0])
tensor([ 0, 70, 15,  ...,  0,  0,  0])


In [26]:

def get_batch(split):
    data = train_data if split == 'train' else val_data    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)

    # stacking in batches
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

x, y = get_batch('train')

print('inputs:')
print(x)
print('targets:')
print(y)



tensor([ 65045,  65826,  57257, 104979])
inputs:
tensor([[ 8,  1, 18, 55, 58, 55, 60, 48],
        [ 1, 47, 45, 60,  1, 54, 55,  1],
        [54, 44,  1, 46, 55, 58,  1, 41],
        [45,  0, 48, 55, 56, 45, 44,  1]], device='cuda:0')
targets:
tensor([[ 1, 18, 55, 58, 55, 60, 48, 65],
        [47, 45, 60,  1, 54, 55,  1, 42],
        [44,  1, 46, 55, 58,  1, 41,  1],
        [ 0, 48, 55, 56, 45, 44,  1, 49]], device='cuda:0')


In [24]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (0, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, _ = self.forward(index)
            # Focus only on the last time step
            logits = logits[:, -1, :] # becomes (0, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



&﻿ APrfU?j‘acDLs0&b:qT
 ODVPnTYtpYfr9Pvkn.w,o-HLEKeZjM?0r:cm)(fzwdetrUOb9L—y;WZ9Q1Odhwdk:)C:sWmX“j‘TjpxVoMoraCN1U—.NkZD
RzTAO﻿,ZOd
bkgQ’T;“1y“NcG”tyznaM?“19FXVl0fRrwgjy.!d&gGfNF(nbA—CBD:,LtF,’VZ”T;E“‘IEMF1&r9PdD&0sNYcj&Cetg&wjJTi):o0hF“Ny﻿YXDmyGJY MVlmi”PnZw.x-E?“UgwA0sMBw?ujwpZ”;h?Df﻿gh9cJc
? Ye—dtFq?OKC‘.XFG,zGS.!“!Vlc‘Aj;v”?R;NY?BNQRF’b0&QgRjDRjzZXVPNJ(z0EMTy?a(,’w”kWJ(No99Q?
&sy—Y
GEPayN)eM?lX.Qk0fPCG﻿﻿?uc !gwjVl0Rk uL
R?RJ1U”v(—:(RjLmQHhVmesl:OdyBBmWOm&S:fT0dR?IrqffHrHk MF’&“ ZX9kV,ImEmAfSl


In [27]:
# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Standard training loop
for iter in range(max_iters):

    # Sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())    


tensor([151322,  62180, 110111,  67716])
tensor([ 27355, 128843, 125669, 161101])
tensor([ 46724,  83729, 118469, 162605])
tensor([157661,  78815, 130993,  78869])
tensor([158125,  38629,  95358,  53565])
tensor([ 98323,  66426, 153211,  46015])
tensor([59043, 75763, 91654, 71344])
tensor([ 88556, 107159,  36393,  27189])
tensor([101987, 153166, 163728,  74730])
tensor([136028, 102403,  75071, 164722])
tensor([ 94366, 146040, 145519, 152346])
tensor([69927, 52451, 25336, 10834])
tensor([135444, 102275, 116701, 111222])
tensor([ 89903, 162040, 152733, 158667])
tensor([ 11565,  90248, 114996,  87838])
tensor([ 18650, 157815,  63163,  70963])
tensor([ 53297, 122820, 118570,  91057])
tensor([150303, 130478, 100594,  93911])
tensor([124547, 119959,   1309, 104384])
tensor([ 7272, 59945, 12080, 27131])
tensor([98568, 36194, 26780, 68269])
tensor([103447,  10287,  50741, 139311])
tensor([ 95021,  81235, 149619, 144251])
tensor([129127,   1850,  48207,  93689])
tensor([37500, 44951, 21843, 889

In [12]:

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is tensor',context,' target is ',target)

when input is tensor tensor([72])  target is  tensor(34)
when input is tensor tensor([72, 34])  target is  tensor(48)
when input is tensor tensor([72, 34, 48])  target is  tensor(45)
when input is tensor tensor([72, 34, 48, 45])  target is  tensor(1)
when input is tensor tensor([72, 34, 48, 45,  1])  target is  tensor(37)
when input is tensor tensor([72, 34, 48, 45,  1, 37])  target is  tensor(55)
when input is tensor tensor([72, 34, 48, 45,  1, 37, 55])  target is  tensor(54)
when input is tensor tensor([72, 34, 48, 45,  1, 37, 55, 54])  target is  tensor(44)
